In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import csv
import time
import random
from collections import defaultdict

columns = defaultdict(list)

with open('C:/Users/KNguyen/NCUC_master_list.csv') as ncuc:
    reader = csv.DictReader(ncuc)
    for row in reader:
        for (k,v) in row.items(): 
            columns[k].append(v)
            
docket_details = columns['Detail']
format_details = [x for x in docket_details if x!='none']

In [4]:
driver = webdriver.Chrome("C:/Users/KNguyen/chromedriver.exe")
with open('C:/Users/KNguyen/NCUC_Documents.csv', 'w', newline='') as ncuc:
    a = csv.writer(ncuc, delimiter=',')
    bill_rows = [['ID', 'Document Type', 'Description', 'Date Filed', 'Link to Files']]
    
    for docket in format_details:
        try:
            driver.get(docket)
            driver.find_element_by_link_text('Documents').click()

            driver.implicitly_wait(1)
            sum_blob = driver.find_element_by_css_selector("#ctl00_ContentPlaceHolder1_PortalPageControl1_ctl38_itemCountLabel")
            sum_clean_blob = sum_blob.text.split()
            num_docs = sum_clean_blob[0]

            if (int(num_docs) > 15):
                new_search = driver.find_element_by_css_selector('#ctl00_ContentPlaceHolder1_PortalPageControl1_ctl38_perPageTextbox')
                new_search.click()
                new_search.send_keys(Keys.BACKSPACE)
                new_search.send_keys(Keys.BACKSPACE)
                driver.implicitly_wait(1)
                new_search.send_keys(num_docs)
                driver.implicitly_wait(2)
                new_search.submit()

            results_dirty = driver.find_element_by_css_selector("#resultsDiv")
            results_html = results_dirty.get_attribute("innerHTML")
            all_docs = results_html.split("<tr")

            docket_id_dirty = driver.find_element_by_class_name("pageTitle").text
            docket_id = docket_id_dirty.replace('Docket ','')

            sample = all_docs[2:]
            for bit in sample:
                format_bit =bit.replace('<td>','<td').replace('</td>','<td').split('<td')

                document_type = format_bit[3]
                #print ("Document Type: " + document_type)

                dirty_desp = format_bit[5].replace('docDescLabel">','</span>').split('</span>')
                description = dirty_desp[1]
                #print ("Description: " + description)

                document_link = ''
                if (len(dirty_desp)==3):
                    document_link = 'Unavailable'
                else:
                    dirty_link = dirty_desp[3].replace('href="','" id').split('" id')
                    if ('confidential' in dirty_link[0]):
                        document_link = 'Confidential'
                    else:
                        document_link = dirty_link[1]
                #print ("Document link: " + document_link)

                dirty_date = format_bit[7].split('>')
                date_filed = dirty_date[1]
                #print ("Date Filed: " + date_filed)

                newrow = [docket_id, document_type, description, date_filed, document_link]
                bill_rows.append(newrow)
                
        except NoSuchElementException:
            driver.refresh()
            continue
            
    a.writerows(bill_rows)
            #print ()